In [4]:
from unified_planning.shortcuts import *

problem = Problem("monkey_and_banana")

# Define types
Location = UserType("Location")
Height = UserType("Height")
Thing = UserType("Thing")

# Define constants/objects
Monkey = Object("Monkey", Thing)
Box = Object("Box", Thing)
Bananas = Object("Bananas", Thing)
A = Object("A", Location)
B = Object("B", Location)
C = Object("C", Location)
Low = Object("Low", Height)
High = Object("High", Height)
problem.add_objects([Monkey, Box, Bananas, A, B, C, Low, High])

# Define fluents
At = Fluent("At", BoolType(), obj=Thing, loc=Location)
AtHeight = Fluent("AtHeight", BoolType(), obj=Thing, h=Height)
On = Fluent("On", BoolType(), obj1=Thing, obj2=Thing)
Have = Fluent("Have", BoolType(), obj1=Thing, obj2=Thing)
Pushable = Fluent("Pushable", BoolType(), obj=Thing)
Climbable = Fluent("Climbable", BoolType(), obj=Thing)
Graspable = Fluent("Graspable", BoolType(), obj=Thing)

# Add fluents to problem
problem.add_fluent(At, default_initial_value=False)
problem.add_fluent(AtHeight, default_initial_value=False)
problem.add_fluent(On, default_initial_value=False)
problem.add_fluent(Have, default_initial_value=False)
problem.add_fluent(Pushable, default_initial_value=False)
problem.add_fluent(Climbable, default_initial_value=False)
problem.add_fluent(Graspable, default_initial_value=False)

# Define initial state
problem.set_initial_value(At(Monkey, A), True)
problem.set_initial_value(At(Bananas, B), True)
problem.set_initial_value(At(Box, C), True)
problem.set_initial_value(AtHeight(Monkey, Low), True)
problem.set_initial_value(AtHeight(Box, Low), True)
problem.set_initial_value(AtHeight(Bananas, High), True)
problem.set_initial_value(Pushable(Box), True)
problem.set_initial_value(Climbable(Box), True)
problem.set_initial_value(Graspable(Bananas), True)

# Define actions

# Go action
go = InstantaneousAction("go", x=Location, y=Location)
go.add_precondition(At(Monkey, go.x))
go.add_effect(At(Monkey, go.y), True)
go.add_effect(At(Monkey, go.x), False)
problem.add_action(go)

# Push action
push = InstantaneousAction("push", obj=Thing, x=Location, y=Location)
push.add_precondition(At(Monkey, push.x))
push.add_precondition(At(push.obj, push.x))
push.add_precondition(Pushable(push.obj))
push.add_precondition(AtHeight(Monkey, Low))
push.add_effect(At(push.obj, push.y), True)
push.add_effect(At(Monkey, push.y), True)
push.add_effect(At(push.obj, push.x), False)
push.add_effect(At(Monkey, push.x), False)
problem.add_action(push)

# ClimbUp action
climb_up = InstantaneousAction("climb_up", obj=Thing, loc=Location)
climb_up.add_precondition(At(Monkey, climb_up.loc))
climb_up.add_precondition(At(climb_up.obj, climb_up.loc))
climb_up.add_precondition(Climbable(climb_up.obj))
climb_up.add_precondition(AtHeight(Monkey, Low))
climb_up.add_effect(On(Monkey, climb_up.obj), True)
climb_up.add_effect(AtHeight(Monkey, High), True)
climb_up.add_effect(AtHeight(Monkey, Low), False)
problem.add_action(climb_up)

# Grasp action
grasp = InstantaneousAction("grasp", obj=Thing, h=Height, loc=Location)
grasp.add_precondition(AtHeight(Monkey, grasp.h))
grasp.add_precondition(AtHeight(grasp.obj, grasp.h))
grasp.add_precondition(At(Monkey, grasp.loc))
grasp.add_precondition(At(grasp.obj, grasp.loc))
grasp.add_precondition(Graspable(grasp.obj))
grasp.add_effect(Have(Monkey, grasp.obj), True)
grasp.add_effect(At(grasp.obj, grasp.loc), False)
grasp.add_effect(AtHeight(grasp.obj, grasp.h), False)
problem.add_action(grasp)

# ClimbDown action
climb_down = InstantaneousAction("climb_down", obj=Thing)
climb_down.add_precondition(On(Monkey, climb_down.obj))
climb_down.add_effect(On(Monkey, climb_down.obj), False)
climb_down.add_effect(AtHeight(Monkey, High), False)
climb_down.add_effect(AtHeight(Monkey, Low), True)
problem.add_action(climb_down)

# Ungrasp action
ungrasp = InstantaneousAction("ungrasp", obj=Thing, h=Height, loc=Location)
ungrasp.add_precondition(Have(Monkey, ungrasp.obj))
ungrasp.add_precondition(At(Monkey, ungrasp.loc))
ungrasp.add_precondition(AtHeight(Monkey, ungrasp.h))
ungrasp.add_effect(Have(Monkey, ungrasp.obj), False)
ungrasp.add_effect(At(ungrasp.obj, ungrasp.loc), True)
ungrasp.add_effect(AtHeight(ungrasp.obj, ungrasp.h), True)
problem.add_action(ungrasp)

# Define goal
problem.add_goal(Have(Monkey, Bananas))

# Solve the problem
with OneshotPlanner(name="pyperplan") as planner:
    result = planner.solve(problem)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Pyperplan returned: %s" % result.plan)
    else:
        print("No plan found.")


  *** Credits ***
  * In operation mode `OneshotPlanner` at line 117 of `/var/folders/q4/2lsmb6qd1ks8137720rg8fz80000gn/T/ipykernel_41422/3920794556.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

Pyperplan returned: SequentialPlan:
    go(A, C)
    push(Box, C, B)
    climb_up(Box, B)
    grasp(Bananas, High, B)
